### 1476行目の下に以下を追加:  
ids = IndexSearch(MAIN_DIR,'abstractL') ids.create_input_textfile(os.path.join(INPUT_DIR,"abstractL_input.txt"))

エラーのため421行目「abst.decode('utf-8')」を「abst」に変更. 

In [1]:
# -*- coding: utf-8 -*-
import re, os, sys, glob, shutil, sqlite3

In [2]:
WS_DIR = '/Users/petadimensionlab/ws/apps/refminer/'
RESOURCE_DIR = os.path.join(WS_DIR,'resources')
MALLET_DIR = '/Users/petadimensionlab/ws/apps/mallet-2.0.8/bin/mallet'

In [3]:
THEME = 'PID'
MAIN_DIR = os.path.join(WS_DIR,THEME)
STAT_DATA_DIR = os.path.join(MAIN_DIR,'stat')
FTS_BASE_DIR = os.path.join(MAIN_DIR)

In [4]:
PUBMED_DIR = os.path.join(WS_DIR,'Pubmed')
sys.path.append(PUBMED_DIR)
from Pubmed_processing import Pubmed

In [5]:
FilReter_DIR = os.path.join(WS_DIR,'FilReter')
DB_BASE_DIR = os.path.join(RESOURCE_DIR,'db')
sys.path.append(FilReter_DIR)
from FilReter import FullTextSearch, DictSearch

In [6]:
MISC_BASE_DIR = os.path.join(DB_BASE_DIR,'misc')
sys.path.append(MISC_BASE_DIR)
#from AhoCorasick import AhoCorasick
#from Inflector import Inflector
from nltk import PorterStemmer

In [7]:
from whoosh.index import create_in, open_dir
from whoosh.fields import *
from whoosh.query import *
from whoosh import qparser
from whoosh.qparser import QueryParser

In [8]:
class IndexSearch(FullTextSearch): #
    def __init__(self,FTS_BASE_DIR,source='abstract'):
        self.FTS_BASE_DIR = FTS_BASE_DIR
        self.DOCS_IDXDIR = os.path.join(self.FTS_BASE_DIR,source+'_idxDB')
        self.read_file_name = os.path.join(MAIN_DIR,source+'.pkl')
    """ save all documents as cPickle (unicode) format """
    def pickle_docs(self,pmid_doc,source):
        import codecs
        import pickle as pickle
        save_file_name = os.path.join(self.FTS_BASE_DIR,source+'.pkl')
        fw = open(save_file_name,'wb')
        try:
            pickle.dump(pmid_doc,fw,protocol=2)#protocol=pickle.HIGHEST_PROTOCOL
        except:
            print ('Failed to save.')
    """ create an input text-file from a pickled file """
    def create_input_textfile(self,input_file):
        import pickle as pickle
        pf = open(self.read_file_name,'rb')
        pmid_abst = pickle.load(pf)
        fw = open(input_file,'w')
        label = 'NA'
        for pmid, abst in pmid_abst.items():
            fw.write(pmid+' '+label+' '+abst+'\n') #.decode('utf-8')
        fw.close()

In [9]:
class AbstractIndexSearch(IndexSearch):
    def __init__(self,FTS_BASE_DIR,source='abstract'):
        self.FTS_BASE_DIR = FTS_BASE_DIR
        self.DOCS_IDXDIR = os.path.join(self.FTS_BASE_DIR,source+'_idxDB')
    """ create index for abstract """    
    def create_indexdb(self,source='abstract'):
        self.DOCS_IDXDIR = os.path.join(self.FTS_BASE_DIR,source+'_idxDB')
        import glob
        import pickle as pickle
        if not os.path.exists(self.DOCS_IDXDIR):
            os.mkdir(self.DOCS_IDXDIR)
        ### create an index database ###
        schema = Schema(title=TEXT(stored=True), path=ID(stored=True), content=TEXT(stored=True))
        ix = create_in(self.DOCS_IDXDIR, schema)
        writer = ix.writer()
        pf = open(os.path.join(self.FTS_BASE_DIR,source+'.pkl'),'rb')
        pmid_doc = pickle.load(pf) 
        for pmid, doc in pmid_doc.items():
            writer.add_document(title=pmid, path=pmid, content=doc)
        os.chdir(self.FTS_BASE_DIR)
        try:
            writer.commit(optimize=True)
        except:
            print ("Failed to write index.")
            writer.cancel()
        ix.close()
    """ create a list of word-frequency """
    def create_VocabDB(self,db_name='Vocab',lemmatization='False'):
        if lemmatization=='False':
            pm = Pubmed(THEME)
            pm.calc_vocab_freq()
            ds = DictSearch(db_name)
            ds.create_simstringdb(db_name,os.path.join(STAT_DATA_DIR,db_name+'.txt'))
        else:
            fr = open(os.path.join(STAT_DATA_DIR,self.db_name+'.txt')).readlines()
            word_lst = []
            for word in fr:
                word = word.replace('\n','')
                word_lst.append(word)
            sew = self.get_stem_equivalent_pair(db_name=db_name)
            fw = open(os.path.join(STAT_DATA_DIR,db_name+'L.txt'),'w')
            for word in word_lst:
                try:
                    pre_word = word
                    word = sew[word]
                    fw.write(word+'\n')
                except:
                    fw.write(word+'\n')
            fw.close()
            vocab_dict_name = THEME+'Lvocabdist'
            ds = DictSearch(vocab_dict_name)
            ds.create_simstringdb(db_name+'L',os.path.join(STAT_DATA_DIR,db_name+'L.txt')) 
    """ get a conversion list of greek unicode to symbol """
    def greek_symbol2name(self):
        fr = open(os.path.join(RESOURCE_DIR,'dict/greek_unicode_utf8literal_symbol.txt')).readlines()
        s2n = {}
        for line in fr:
            line = line.replace('\n','')
            uni, sym, utf8, name = line.split('\t')
            s2n[sym] = name
        return s2n    
    """ get a pair of stem equivalent words """
    def get_stem_equivalent_pair(self,db_name='Vocab',save2file='False'):
        ds = DictSearch(db_name)
        ps = PorterStemmer() 
        fr = open(os.path.join(STAT_DATA_DIR,db_name+'.txt')).readlines()
        word_lst = []
        for word in fr:
            word = word.replace('\n','')
            word_lst.append(word)
        sim_lst = []
        for item in word_lst:
            #db.threshold = 1.0-1.0/(len(item)+1) # theshold to obtain a unique return
            #print( item )
            res = ds.single_db_retrieve(item,threshold=0.8)
            n = len(res)
            if n==2:
                sim_lst.append(res)
            elif n>2:
                head = res[0]
                for i in range(1,n):
                    sim_lst.append([res[0],res[i]])
        ## get stem equivalent words ##
        common_sfx_lst = []
        fr = open(os.path.join(RESOURCE_DIR,'dict/common_suffix.txt')).readlines()
        for item in fr:
            item = item.replace('\n','')
            common_sfx_lst.append(item)
        sew = {}
        for lst in sim_lst:
            smw0 = ps.stem(str(lst[0])) #len(str(lst[0]))-1
            smw1 = ps.stem(str(lst[1]))
            if smw0==smw1: # stem-equivalent
                if len(str(lst[0]))>=len(str(lst[1])):
                    sfx = str(lst[0]).replace(str(lst[1]),'')
                    if sfx in common_sfx_lst:
                        sew[lst[0]] = lst[1]
                else:
                    sfx = str(lst[1]).replace(str(lst[0]),'')
                    if sfx in common_sfx_lst:
                        sew[lst[1]] = lst[0]
        if not save2file=='False':
            fw = open(os.path.join(STAT_DATA_DIR,self.header+'_stem_equivalence.txt'),'w')
            for key, val in sew.items():
                fw.write(key+'\t'+val+'\n')
            fw.close()
        return sew
    """ lemmatize documents """
    def lemmatize_docs(self,source='abstract'):
        import pickle as pickle
        vocab_name = THEME+'Vocab.txt'
        vocab_db_name = THEME+'Vocab'
        #if not os.path.exists(os.path.join(STAT_DATA_DIR,vocab_name)):
        self.create_VocabDB(db_name=vocab_db_name,lemmatization='False')
        pm = Pubmed(THEME)
        #import pdb;pdb.set_trace()
        pmid_doc = pm.collect_info('abstract')
        gs2n = self.greek_symbol2name()
        symbol_lst = gs2n.keys()
        ## delete NO-abstract documents && relpace greek-words ##
        for pmid, doc in pmid_doc.copy().items():
            if doc == 'No abstract':
                del pmid_doc[pmid]
            else:
                for sym in symbol_lst:
                    match = doc.find(sym)
                    if match>=0:
                        doc.replace(sym,gs2n[sym]) #sym.decode('utf-8')
        if not os.path.exists(self.DOCS_IDXDIR):
            self.pickle_docs(pmid_doc,source)
            self.create_indexdb(source=source)
        fts = AbstractIndexSearch(MAIN_DIR,source)
        sew = self.get_stem_equivalent_pair(db_name=vocab_db_name)
        search_words = sew.keys()
        for word in search_words:
            pmids = fts.matched_docs(word)
            for pmid in pmids:
                try:
                    doc = pmid_doc[pmid]
                    Ldoc = doc.replace(word,sew[word])
                    pmid_doc[pmid] = Ldoc #unicode(new_abst)
                except:
                    pass
        self.pickle_docs(pmid_doc,source+'L')
        self.create_indexdb(source=source+'L')
    """ send a query to reference DB """
    def query2refdb(self,qtype,query,target='all'):
        refdb_name = THEME+'Ref.db'
        connection = sqlite3.connect(os.path.join(SAVE_DATA_DIR,refdb_name))
        cursor = connection.cursor()
        """ pubmed_id, pub_type, title, authors, journal, year, vol, issue, pages, abstract """
        sql = "SELECT %s FROM tbl WHERE %s='%s';" % (target,qtype,query)
        cc = cursor.execute(sql)
        tbl = []
        for row in cc:
            tbl.append(row[0])
        cursor.close()
        return tbl[0]
    """ check the correponding abstract in which the retrieved word appears """
    def check_abst(self,querystring,search_field='content',qlim=5,write_as_html='True'):
        flag = 0
        pmid_lst = self.matched_docs(querystring=querystring,search_field=search_field,qlim=qlim)
        if len(pmid_lst)==0:
            flag = 1
        tagged_query = '<font color="red"><b>'+querystring+'</b></font>'
        pmid_abst = {}
        for pmid in pmid_lst:
            title = self.query2refdb(qtype='pubmed_id',query=pmid,target='title')
            pub_type = self.query2refdb(qtype='pubmed_id',query=pmid,target='pub_type')
            journal = self.query2refdb(qtype='pubmed_id',query=pmid,target='journal')
            year = self.query2refdb(qtype='pubmed_id',query=pmid,target='year')
            abst = self.query2refdb(qtype='pubmed_id',query=pmid,target='abstract')
            abst = abst.replace(querystring,tagged_query)
            ref = '|'.join([title,journal,str(year),pub_type,abst])
            pmid_abst[pmid] = ref
        if write_as_html=='True':
            if flag==0:
                save_file_name = os.path.join(os.path.join(MAIN_DIR,'abst_check'),querystring+'.html')
            else:
                save_file_name = os.path.join(os.path.join(MAIN_DIR,'abst_check/noabst'),querystring+'.html')
            fw = open(save_file_name,'w')
            fw.write('<html><body>'+'\n')
            for pmid, ref in pmid_abst.items():
                title,journal,year,pub_type,abst = ref.split('|')
                fw.write('<h3>'+title+'</h3>'+'\n')
                fw.write('<h4><it>'+journal+'</it> ('+year+') ['+pub_type+'] [pmid: <a href="http://www.ncbi.nlm.nih.gov/pubmed?term='+pmid+'" target=blank>'+pmid+'</a>]</h4>'+'\n')
                fw.write('<p>'+abst+'</p>'+'\n')
            fw.write('</body></html>')
            fw.close()

In [10]:
import pickle as pickle
class LDA():
    """ Latent Dirichlet Allocation by MALLET """
    def __init__(self,source='abstract'):
        self.LDA_DIR = os.path.join(MAIN_DIR,'LDA')
        self.header = THEME
        self.source = source
        self.MALLET = os.path.join(MALLET_DIR)
        self.input_file = os.path.join(MAIN_DIR,self.source+'_input.txt')
        self.train_file = os.path.join(self.LDA_DIR,self.header+self.source+'.mallet')
        self.stopwords_file = os.path.join(os.path.join(RESOURCE_DIR,'stopwords'),'stopwords_all.txt')
        self.ostate_zip = os.path.join(self.LDA_DIR,self.header+self.source+'_state.gz')
        self.ostate = os.path.join(self.LDA_DIR,self.header+self.source+'_state.txt')
        #odoc = os.path.join(LDA_OUTPUT_DIR,self.header+'_docs.txt')
        self.okey = os.path.join(self.LDA_DIR,self.header+self.source+'_keys.txt')
        #self.nodesfreqint = os.path.join(self.LDA_DIR,self.header+self.source+'_nodesfreqint.txt')
    """ execute LDA """        
    def exec_LDA(self,num=-1,parms={'alpha':1.0,'beta':0.1,'num-topics':50,'num-threads':8,'num-top-words':20,'num-iterations':1000,'train':'on'}):
        import gzip
        #if not os.path.exists(self.input_file):
        #    adis = IndexSearch(MAIN_DIR,self.source)
        #    adis.create_input_textfile(input_file=self.input_file)
        if parms['train']=='on':
            exe1 = '%s import-file --input %s --output %s --keep-sequence --preserve-case --print-output FALSE --stoplist-file %s' % (self.MALLET, self.input_file, self.train_file, self.stopwords_file)
            #print(exe1)
            os.system(exe1)
        if not num==-1:
            self.okey = os.path.join(self.LDA_DIR,self.header+self.source+'_keys'+str(num)+'.txt')
        exe2 = '%s train-topics --input %s --output-state %s --output-topic-keys %s --alpha %f --beta %f --num-topics %d --num-threads %d --num-top-words %d --num-iterations %d' % (self.MALLET,self.train_file,self.ostate_zip,self.okey,parms['alpha'],parms['beta'],parms['num-topics'],parms['num-threads'],parms['num-top-words'],parms['num-iterations'])
        #print(exe2)
        os.system(exe2)
        fr = open(self.input_file, 'rb').readlines()
        pmid_lst = []
        for line in fr:
            lst = line.decode().split(' ')
            pmid_lst.append(lst[0])
        f = gzip.open(self.ostate_zip, 'rb')
        file_content = f.readlines()
        f.close()
        fr = []
        for i in range(3,len(file_content)):
            fr.append( file_content[i].decode('utf-8') )
        #fr = file_content[3:len(file_content)]        
        docnum = 0
        fw = open(self.ostate,'w')
        for line in fr:
            lst = line.split(' ')
            pmid = lst[1]
            word = lst[4]
            topicnum = lst[5]
            if lst[0] == str(docnum):
                pmid = pmid_lst[docnum]
                fw.write(pmid+'\n')
                docnum += 1
            fw.write(word+'\t'+topicnum)
        fw.close()
        os.remove(self.ostate_zip)
        os.remove(self.train_file)
    """ return a reverse combined key """
    def get_reversekey(self,fkey):
        lst = fkey.split('__')
        rkey = lst[1]+'__'+lst[0]
        return rkey
    """ check whether a key exists or not """
    def is_keyexists(self,fkey,dic):
        flag = 0
        chk = dic.get(fkey,'NotExist')
        if chk=='NotExist': # no-fkey
            rkey = self.get_reversekey(fkey)
            chk = dic.get(rkey,'NotExist')
            if chk=='NotExist': # no rkey (and fkey)
                dic[fkey] = 1
            else: # rkey exists
                val = dic[rkey]
                val += 1
                dic[rkey] = val
        else: # fkey exists
            val = dic[fkey]
            val += 1
            dic[fkey] = val
    """ remove self-referred node """
    def remove_selfref(self,dic):
        remkeys = []
        for key in dic.keys():
            key1, key2 = key.split('__')
            if key1==key2:
                remkeys.append(key)
        for remkey in remkeys:
            del dic[remkey]
        return dic
    """ get all possible pairs from a list """
    def gen_fkeys(self,wlst):
        n = len(wlst)
        fkey_lst = []
        for i in range(n):
            for j in range(i,n):
                fkey = wlst[i]+'__'+wlst[j]
                fkey_lst.append(fkey)
        return fkey_lst
    """ convert a pickle file to readable file format """
    def conv_pickle2txt(self,treeid):
        pf =  open(os.path.join(self.LDA_DIR,self.header+self.source+'_com_'+treeid+'.pkl'),'rb')
        nodes_freq = pickle.load(pf)
        save_file_name = os.path.join(self.LDA_DIR,self.header+self.source+'_com_'+treeid+'.txt')
        fw = open(save_file_name,'w')
        for key, val in sorted(nodes_freq.items(), key=lambda x:x[1], reverse=True):
            lst = key.split('__')
            fw.write('\t'.join(lst)+'\t'+str(val)+'\n')
        fw.close()
    """ convert a pickle file to readable file format """
    def conv_txt2pickle(self,treeid):
        fr =  open(os.path.join(self.LDA_DIR,self.header+self.source+'_com_'+treeid+'.txt')).readlines()
        nodes_freq = {}
        for line in fr:
            node1, node2, freq = line.split('\t')
            key = node1+'__'+node2
            nodes_freq[key] = int(freq)
        save_file_name = os.path.join(self.LDA_DIR,self.header+self.source+'_com_'+treeid+'.pkl')
        fw = open(save_file_name,'w')
        pickle.dump(nodes_freq,fw)
        fw.close()
    """ check convergence (not convergent if there exists a key with the freqency less than a given threshold) """
    def check_convergence(self,nodes_freq,thr=1):
        flag = 0
        freq_lst = [count for count in nodes_freq.values() if count<=thr]
        if len(fre_lst)>0:
            flag = 1
        return flag
    """ get edge counts """
    def get_nodes_counts(self,nodes_freq,save2file='False'):
        fr = open(self.okey).readlines()
        for line in fr:
            line = line.replace('\n','')
            num1, num2, words = line.split('\t')
            wlst = words.split(' ')
            nullidx = wlst.index('')
            wlst.pop(nullidx)
            fkey_lst = self.gen_fkeys(wlst)
            for fkey in fkey_lst:
                self.is_keyexists(fkey,nodes_freq)
        nodes_freq = self.remove_selfref(nodes_freq)
        if not save2file=='False':
            save_file_name = os.path.join(self.LDA_DIR,self.header+self.source+'_com_d0.pkl')
            fw = open(save_file_name,'wb')
            pickle.dump(nodes_freq,fw)
            fw.close()
            self.conv_pickle2txt('d0')            
        return nodes_freq
    """ sequencial LDA """
    def seq_LDA(self,iter=10,convergence_check='False'):
        nodes_freq = {}
        for i in range(1,iter+1):
            self.exec_LDA(num=i)
            nodes_freq = self.get_nodes_counts(nodes_freq=nodes_freq,save2file='True')
        if not convergence_check=='False':
            chk = self.check_convergence(nodes_freq,thr=1)
            count = itr
            while chk==1:
                nodes_freq = self.get_nodes_counts(nodes_freq=nodes_freq,save2file='True')
                chk = self.check_convergence(nodes_freq,thr=1)
                count += 1
            print('convergence after %d computations.' % (count))
    """ get nodes and their weight as lines """
    def conv2edge_weight_list(self,nodes_freq,lthr=1,hthr='infty'):
        wedge_lst = []
        for key, freq in nodes_freq.items():
            node1, node2 = key.split('__')
            if hthr=='infty':
                if freq>=lthr:
                    wedge_lst.append('\t'.join([node1,node2,str(freq)]))
            else:
                if freq>=lthr and freq<=hthr:
                    wedge_lst.append('\t'.join([node1,node2,str(freq)]))
        return wedge_lst
    """ extract community structures """
    def detect_communities(self,nodes_freq,treeid,lthr=1,hthr='infty',save2file='False',draw='False'):
        SCRPT_BASE_DIR = os.path.join(RESOURCE_DIR,'scripts')
        sys.path.append(SCRPT_BASE_DIR)
        import community
        import networkx as nx
        nodes_lst = nodes_freq.keys()
        wedge_lst = self.conv2edge_weight_list(nodes_freq,lthr=lthr,hthr=hthr)
        G = nx.parse_edgelist(wedge_lst,nodetype=str, data=(('weight',int),))
        partition = community.best_partition(G) #first compute the best partition
        ##import louvain
        ##import igraph
        ##snodes = set()
        ##for key in nodes_lst:
        ##    item1,item2 = key.split('__')
        ##    snodes.add(item1)
        ##    snodes.add(item2)
        ##nodes = list(snodes)
        ##edges = []
        ##for item in wedge_lst:
        ##    n1,n2,freq = item.split('\t')
        ##    n1idx = nodes.index(n1)
        ##    n2idx = nodes.index(n2)
        ##    edges.append((n1idx,n2idx))
        ##G = igraph.Graph(vertex_attrs={"label": nodes},edges=edges)
        ##partition = louvain.find_partition(G);
        ## get community-network ##
        baseid = treeid
        for com in range(0,len(set(partition.values()))):
            wlst = [node for node in partition.keys() if partition[node]==com]
            sub_nodes_freq = {}
            for word in wlst:
                hitnodes = [nodes for nodes in nodes_lst if word in nodes]
                for hitnode in hitnodes:
                    node1, node2 = hitnode.split('__')
                    if node1 in wlst and node2 in wlst:
                        sub_nodes_freq[hitnode] = nodes_freq[hitnode]
            treeid = baseid+'d'+str(com)
            save_file_name = os.path.join(self.LDA_DIR,self.header+self.source+'_com_'+treeid+'.pkl')
            fw = open(save_file_name,'wb')
            pickle.dump(sub_nodes_freq,fw)
            fw.close()
            self.conv_pickle2txt(treeid)
        if not draw=='False':
            import matplotlib.pyplot as plt
            #drawing
            size = float(len(set(partition.values())))
            pos = nx.spring_layout(G)
            count = 0.0
            for com in set(partition.values()):
                count += 1.0
                list_nodes = [nodes for nodes in partition.keys() if partition[nodes]==com]
                nx.draw_networkx_nodes(G,pos,list_nodes,node_size=20,node_color=str(count/size))
            nx.draw_networkx_edges(G,pos,alpha=0.5)
            plt.show()
        #return len(set(partition.values()))       
    """ get hierarchical topics with the given depth """
    def get_hierarchical_topics(self,maxdepth=1):
        import glob
        REPdepth = re.compile('d\d+')
        main_dir = os.getcwd()
        for d in range(0,maxdepth):
            os.chdir(self.LDA_DIR)
            com_lst = glob.glob(self.header+self.source+'_com_*.pkl')
            depth_lst = []
            for item in com_lst:
                item = item.replace(self.header+self.source+'_com_','')
                treeid = item.replace('.pkl','')
                lst = REPdepth.findall(treeid)
                if len(lst)==d+1:
                    depth_lst.append(treeid)   
            os.chdir(main_dir)
            for treeid in depth_lst:
                read_file_name = self.header+self.source+'_com_'+treeid+'.pkl'
                pf = open(os.path.join(self.LDA_DIR,read_file_name),'rb')
                nodes_freq = pickle.load(pf)
                self.detect_communities(nodes_freq,treeid,lthr=1,hthr='infty',save2file='True',draw='False')
    """ print a list of words for each tree-branch """
    def get_topicwords(self,treeid):
        pf =  open(os.path.join(self.LDA_DIR,self.header+self.source+'_com_'+treeid+'.pkl'),'rb')
        nodes_freq = pickle.load(pf)
        key_lst = nodes_freq.keys()
        wlst = []
        for key in key_lst:
            node1, node2 = key.split('__')
            wlst.append(node1)
            wlst.append(node2)
        wlst = list(set(wlst))
        wlst.sort()
        return wlst
    """ get all tree information """
    def get_all_topicwords(self):
        import glob
        REPdn = re.compile('d\d+')
        main_dir = os.getcwd()
        os.chdir(self.LDA_DIR)
        com_lst = glob.glob(self.header+self.source+'_com_*.pkl')
        root_idx = com_lst.index(self.header+self.source+'_com_d0.pkl')
        com_lst.pop(root_idx)
        com_lst.sort()
        fw = open(os.path.join(self.LDA_DIR,self.header+self.source+'_comwise_words.txt'),'w')
        for item in com_lst:
            item = item.replace(self.header+self.source+'_com_','')
            treeid = item.replace('.pkl','')
            fw.write(treeid+'\n')
            wlst = self.get_topicwords(treeid)
            fw.write(' '.join(wlst)+'\n')
        fw.close()
        dns_set = set()
        for item in com_lst:
            item = item.replace(self.header+self.source+'_com_','')
            treeid = item.replace('.pkl','')
            dns = REPdn.findall(treeid)
            dns_set.add(len(dns))
        maxdepth = max(list(dns_set))
        fw = open(os.path.join(self.LDA_DIR+'/'+self.header+'_maxdepth_words.txt'),'w')
        for item in com_lst:
            item = item.replace(self.header+self.source+'_com_','')
            treeid = item.replace('.pkl','')
            dns = REPdn.findall(treeid)
            if len(dns)==maxdepth:
                fw.write(treeid+'\n')
                wlst = self.get_topicwords(treeid)
                fw.write(' '.join(wlst)+'\n')
        fw.close()

In [11]:
class make_cattree():
    def __init__(self,source='abstractL'):
        self.source = source
        self.REPcat = re.compile('(d\d{1,2})+$')
        self.REP1stline = re.compile('all_stemcat\[d0\]')
        self.LDA_DIR = os.path.join(MAIN_DIR,'LDA')
        self.id_name = {}
    def generate_space(self,depth=3):
        sp = ''
        for i in range(4*(depth-1)):
            sp = sp+' '
        return sp
    def generate_lowest_part(self,depth=3):
        s = ''
        sp = self.generate_space(depth)
        tmp = 'depth%d = all_stemcat[d%d]\n' % (depth,depth-1)
        s = s+sp+tmp
        tmp = 'depth%d.sort()\n' % (depth)
        s = s+sp+tmp
        tmp = 'for d%d in depth%d:\n' % (depth,depth)
        s = s+sp+tmp
        tmp = '    fw.write(cat_words[d%d])\n' % (depth)
        s = s+sp+tmp
        return s
    def create_recursive_tree(self,depth):
        html_file_name = os.path.join(self.LDA_DIR,self.source)
        body = 'fw = open(\'%s_tree.html\',\'w\')\n' % (html_file_name)
        for i in range(1,depth):
            s = ''
            sp = self.generate_space(i)
            tmp = 'depth%d = all_stemcat[d%d]\n' % (i,i-1)
            s = s+sp+tmp
            tmp = 'depth%d.sort()\n' % (i)
            s = s+sp+tmp
            tmp = 'for d%d in depth%d:\n' % (i,i)
            s = s+sp+tmp
            tmp = '    try:\n'
            s = s+sp+tmp
            tmp = '        name = id_name[d%d]\n' % (i)
            s = s+sp+tmp
            tmp = '    except:\n'
            s = s+sp+tmp
            tmp = '        name = d%d\n' % (i)
            s = s+sp+tmp
            #tmp = '    s = \'<li><span class=\"topic%d\">\%s</span>\n\' \% (name)' % (i)
            tmp = '    s = \'<li><span class="topic%d">%ss</span>\\n\' %s (name)\n' % (i,"%","%")
            s = s+sp+tmp
            tmp = '    fw.write(s)\n'
            s = s+sp+tmp
            tmp = '    fw.write(\'<ul>\\n\')\n'
            s = s+sp+tmp
            body = body+s
        s = self.generate_lowest_part(depth)
        body = body+s
        tail = ''
        for i in range(1,depth):
            sp = self.generate_space(depth-i+1)
            tmp = 'fw.write(\'</ul></li>\\n\')\n'
            tail = tail+sp+tmp
        body = body+tail+'fw.close()\n'
        body = self.REP1stline.sub('[stem for stem in all_stemcat.keys() if get_depth(stem)==1]',body)
        return body
    def create_exec_script(self,depth):
        body = self.create_recursive_tree(depth)
        fr = open(os.path.join(WS_DIR,'make_cattree_head.py')).read()
        fw = open('make_cattree.py','w')
        fw.write(fr+body)
        fw.close()
    def create_tree_html(self,depth):
        self.create_exec_script(depth)
        cmd = 'python make_cattree.py %s %s %s' % (THEME,self.LDA_DIR,depth)
        #print(cmd)
        os.system(cmd)
    def concatenate_html(self,jp='True'):
        template_dir = os.path.join(WS_DIR,'template/')
        head_file_name = os.path.join(template_dir,'head.html')
        head = open(head_file_name).read()
        body_file_name = os.path.join(self.LDA_DIR,self.source+'_tree.html')
        body = open(body_file_name).read()
        tail_file_name = os.path.join(template_dir,'tail.html')
        tail = open(tail_file_name).read()
        fw = open(body_file_name,'w')
        fw.write(head+'\n'+body+'\n'+tail)
        fw.close()
        if jp=='True':
            template_dir = os.path.join(WS_DIR,'template')
            head_file_name = os.path.join(template_dir,'literature_html_headJP.html')
            head = open(head_file_name).read()
            body_file_name = os.path.join(self.LDA_DIR,self.source+'_treeJP.html')
            body = open(body_file_name).read()
            tail_file_name = os.path.join(template_dir,'literature_html_tail.html')
            tail = open(tail_file_name).read()
            fw = open(body_file_name,'w')
            fw.write(head+'\n'+body+'\n'+tail)
            fw.close()
    def conv2jp(self):
        ds = DictSearch()
        read_file_name = os.path.join(self.LDA_DIR,self.source+'_tree.html')
        fr = open(read_file_name).readlines()
        save_file_name = os.path.join(self.LDA_DIR,self.source+'_treeJP.html')
        fw = open(save_file_name,'w')
        for line in fr:
            tmp = self.REPlihead.sub('',line)
            lst = self.REPareftail.split(tmp)
            for item in lst:
                if self.REParefhead.search(item):
                    word = self.REParefhead.sub('',item)
                    word_pair = ds.translate2jp(word)
                    if word_pair[1]=='Not Found':
                        word_pair = ds.translate2jp(word.lower())
                    if not word_pair[1]=='Not Found':
                        jp_rep_word = word_pair[1].split(', ')[0]
                        rm_lst = self.REPkakko.findall(jp_rep_word)
                        for rm in rm_lst:
                            jp_rep_word = jp_rep_word.replace(rm,'')
                        line = line.replace(word_pair[0],jp_rep_word+' ['+word_pair[0]+']')
            fw.write(line.encode('utf-8'))
        fw.close()

In [12]:
if __name__=='__main__':
    """ RefMiner """
    #pm = Pubmed(THEME)
    """ full text search (create db) """
    ais = AbstractIndexSearch(MAIN_DIR,'abstractL')
    ais.lemmatize_docs()
    adis = IndexSearch(MAIN_DIR,'abstractL')
    adis.create_input_textfile(input_file=os.path.join(MAIN_DIR,"abstractL_input.txt"))
    """ LDA """
    ## Error handling while performing mallet: certain machine symbols may cause error. see below: 
    ## https://stackoverflow.com/questions/21417809/error-while-importing-txt-file-into-mallet##
    cmd = 'mv %s/abstractL_input.txt %s/abstractL_input_original.txt' % (MAIN_DIR,MAIN_DIR)
    os.system(cmd)
    #print( cmd )
    cmd = 'tr -dc [:alnum:][\ ,.]\\n < %s/abstractL_input_original.txt > %s/abstractL_input.txt' % (MAIN_DIR,MAIN_DIR)
    os.system(cmd)
    #print( cmd )
    lda = LDA(source='abstractL')
    lda.seq_LDA(iter=1)
    lda.get_hierarchical_topics(maxdepth=4)
    lda.get_all_topicwords()
    """ Tree structure """
    tree = make_cattree(source='abstractL')
    tree.create_tree_html(depth=4)
    tree.concatenate_html(jp='False')

/Users/petadimensionlab/ws/apps/refminer/retriever /Users/petadimensionlab/ws/apps/refminer/resources/FilReter/db/dict/PIDVocab/PIDVocab.db 0.7 abcdefg
/Users/petadimensionlab/ws/apps/refminer/retriever /Users/petadimensionlab/ws/apps/refminer/resources/FilReter/db/dict/PIDVocab/PIDVocab.db 0.7 abcdefg


FileNotFoundError: [Errno 2] No such file or directory: '/Users/petadimensionlab/ws/apps/refminer/PID/LDA/PIDabstractL_state.gz'